# Analysis and Visualization of Complex Agro-Environmental Data
---
## Outlier analysis

This refers to the analytical and graphical processes of identifying and examining data observations that in some way significantly differ from the rest of the dataset. This can be a deviation in terms of values that go much beyond the majority of values of a given variable (univariate outlier) or a combination of values that differ from a given pattern or trend in data (multivariate outlier).


Import modules:

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as sts
import seaborn as sns
import matplotlib.pyplot as plt
import statistics as stat

### 1. Example of the impact of outliers on descriptive statistics

In the following simple example, let's check the impact of including a single oulier in a dataset on the most common univariate descriptive statistics.

In [ ]:
# no outlier
v1 = [1,3,5,2,5,6,8,3,6] # list
v1 = pd.Series(v1) # pandas series
medianv1 = pd.Series({'median': v1.median()})
statv1 = v1.describe()
table1 = pd.concat([medianv1, statv1])

# add outlier (ex. 200)
v2 = [1,3,5,2,5,6,8,3,6,200] # list with outlier
v2 = pd.Series(v2) # pandas series
medianv2 = pd.Series({'median': v2.median()}) # compute median
statv2 = v2.describe() # compute summary statistics table
table2 = pd.concat([medianv2, statv2]) # concatenate series

pd.DataFrame({'No outlier': table1, 'With outlier': table2}) # combine resulting series into a single data frame.

As you can see, the mean and standard deviation changed drastically after adding the outlier. The median and percentiles did not change (in this example). In fact, these location measures are less sensitive to the presence of outliers.

### 2. Detection of outliers

Let's have a look at the dataset `winequality_red.csv` available [here](https://www.kaggle.com/datasets/sgus1318/winedata?resource=download) with 12 variables (columns) related with wine quality characteristics. 


In [ ]:
df_wine = pd.read_csv('winequality_red.csv')
df_wine.head()

In [ ]:
# Check the size of the dataset
len(df_wine)

#### 2.1 Tukey's Interquartile Range method

Let's first run a univariate scatterplot of the variable 'pH' as well as a boxplot:

In [ ]:
g = sns.JointGrid(data=df_wine, x=df_wine.index, y='pH')
g.plot(sns.scatterplot, sns.boxplot)
g.ax_marg_x.set_axis_off() # removes the marginal x-axis
g.ax_marg_y.set_axis_off() # removes the marginal y-axis
g.ax_marg_x.remove() # removes the marginal x boxplot (index of each observation)
g.set_axis_labels(xlabel='Observation index', ylabel='pH')
g.fig.set_figwidth(8) # to define the width to get a non-square figure with JointGrid 
g.fig.set_figheight(4) # to define the height to get a non-square figure with JointGrid 
plt.show()

This plot shows several outliers in the variable 'pH'. Boxplots are based on the concept of Interquartile Range (IQR) that is used in statistics to measure the statistical dispersion and data variability by dividing the dataset into quartiles. The first, second and fourth quartiles divides the data into four intervals of equal probability: <Q1; Q1-Q2; Q2-Q3; >Q3.

IQR is the difference between the third quartile and the first quartile (IQR = Q3 -Q1). Outliers using this criteria are defined as ***the observations that are below Q1 − 1.5 x IQR (i.e. defining the lower whisker), or above Q3 + 1.5 x IQR (i.e. defining the upper whisker)***.

We can run some code to try to count and detect these outliers:

In [ ]:
# function to compute the IQR (upper and lower whisker)

def out_iqr(df , column):
    global loweriqr,upperiqr
    q25, q75 = np.quantile(df[column], 0.25), np.quantile(df[column], 0.75)
    # calculate the IQR
    iqr = q75 - q25
    # calculate the outlier cutoff
    cut_off = iqr * 1.5
    # calculate the lower and upper bound value
    loweriqr, upperiqr = q25 - cut_off, q75 + cut_off
    # print results
    print('IQR = ',iqr)
    print('lower whisker = ', loweriqr)
    print('upper whisker = ', upperiqr)
    # Calculate the number of records below and above lower and above bound value respectively
    df1 = df[df[column] > upperiqr]
    df2 = df[df[column] < loweriqr]
    return print('Total number of outliers = ', df1.shape[0]+ df2.shape[0])

In [ ]:
# Get the IQR limits ad the number of outliers
out_iqr(df_wine,'pH')

In [ ]:
# histogram with areas defining outlier 
plt.figure(figsize = (10,6))
sns.set_style('darkgrid') # set the background to grey with white tckmarks (ggplot default style)
sns.histplot(df_wine.pH,
    color='skyblue',
    edgecolor='none'
    )
plt.axvspan(xmin= df_wine.pH.min(), xmax = loweriqr, alpha=0.2, color='red') # lower was defined by the function 'out_iqr' produced above
plt.axvspan(xmin = upperiqr,xmax= df_wine.pH.max(),alpha=0.2, color='red') # upper was defined by the function 'out_iqr' produced above
plt.show()

In [ ]:
# To go back to matplotlib default style run:
plt.style.use('default')

In [ ]:
# Get the position (index) of the outliers in the dataset
outlier_indices = np.where((df_wine['pH'] > upperiqr) | (df_wine['pH'] < loweriqr))
print(outlier_indices)
print(np.size(outlier_indices)) # get the number of outliers

In [ ]:
# add a column to identify the IQR-based outliers
df_wine['Outlier_iqr'] = (df_wine['pH'] < loweriqr) | (df_wine['pH'] > upperiqr)
pd.crosstab(df_wine['Outlier_iqr'], columns='count')

In [ ]:
# Visualize the univariate ouliers of the dataset
g = sns.JointGrid(data=df_wine, x=df_wine.index, y='pH', hue='Outlier_iqr')
g.plot(sns.scatterplot, sns.boxplot)
g.ax_marg_x.set_axis_off() # removes the marginal x-axis
g.ax_marg_y.set_axis_off() # removes the marginal y-axis
g.ax_marg_x.remove() # removes the marginal x boxplot (index of each observation)
g.set_axis_labels(xlabel='Observation index', ylabel='pH')
g.fig.set_figwidth(8) # to define the width to get a non-square figure with JointGrid 
g.fig.set_figheight(4) # to define the height to get a non-square figure with JointGrid 
plt.show()

In [ ]:
# Remove outliers and create new data frame
df_new = df_wine.loc[(df_wine['pH'] < upperiqr) & (df_wine['pH'] > loweriqr)]
len(df_new)

In [ ]:
# Check the impact on univariate summary statstics of removing outliers
# original dataset:
median_pH = pd.Series({'median': df_wine['pH'].median()})
stat_pH = df_wine['pH'].describe()
table1 = pd.concat([median_pH, stat_pH])
# no outliers:
median_pH2 = pd.Series({'median': df_new['pH'].median()})
stat_pH2 = df_new['pH'].describe()
table2 = pd.concat([median_pH2, stat_pH2])

pd.DataFrame({'Original': table1, 'No outliers': table2})

#### 2.2 Standard deviation method

This method uses the standard deviation of the sample as a cut-off for identifying outliers and applies only to data with Gaussian-like distributions,

As we saw, if a data distribution is normal then approximately:

* 68% of the data values lie within one standard deviation of the mean
* 95% are within two standard deviations
* 99.7% lie within three standard deviations.

Outliers may be defined acording to a cutoff value corresponding to 2 times stdev or 3 times stdev.
The code below follows an approach similar to the previous method:

In [ ]:
# function to compute detect values below mean - 3xstdev or above mean + 3xstdev that will define the outliers

def out_std(df, column):
    global lowersd, uppersd
    # calculate the mean and standard deviation of the data frame
    data_mean, data_std = df[column].mean(), df[column].std()
    # calculate the cutoff value
    cut_off = data_std * 3
    # calculate the lower and upper bound value
    lowersd, uppersd = data_mean - cut_off, data_mean + cut_off
    print('The lower bound value is', lowersd)
    print('The upper bound value is', uppersd)
    # Calculate the number of records below and above lower and above bound value respectively
    df1 = df[df[column] > uppersd]
    df2 = df[df[column] < lowersd]
    return print('Total number of outliers are', df1.shape[0]+ df2.shape[0])

In [ ]:
# Get the 3xSD limits ad the number of outliers
out_std(df_wine,'pH')

In [ ]:
# add a column to identify the SD-based outliers
df_wine['Outlier_sd'] = (df_wine['pH'] < lowersd) | (df_wine['pH'] > uppersd)
pd.crosstab(df_wine['Outlier_sd'], columns='count')

In [ ]:
# Visualize the univariate ouliers of the dataset
g = sns.JointGrid(data=df_wine, x=df_wine.index, y='pH', hue='Outlier_sd')
g.plot(sns.scatterplot, sns.boxplot)
g.ax_marg_x.set_axis_off() # removes the marginal x-axis
g.ax_marg_y.set_axis_off() # removes the marginal y-axis
g.ax_marg_x.remove() # removes the marginal x boxplot (index of each observation)
g.set_axis_labels(xlabel='Observation index', ylabel='pH')
g.fig.set_figwidth(8) # to define the width to get a non-square figure with JointGrid 
g.fig.set_figheight(4) # to define the height to get a non-square figure with JointGrid 
plt.show()

This method tends to select fewer outliers. Because outliers increase the standard deviation, the method may fail to detect outliers.

#### 2.3 Z-score method

Z-score describes the position of a raw score in terms of its distance from the mean, when measured in standard deviation units. This method also assumes a Gaussian distribution of the data. The outliers will be defined as the data points that are located in the tails of the distribution.

`Z_score = (Xi - mean) / standard deviation`

where:
* Xi - observation
* 'mean' - mean of all Xi 
* 'standard deviation' - standard deviation of all Xi

An outlier is then a normalized data point which has an absolute value greater than Zthr. That is:
|Z_score| > Zthr

Commonly used Zthr values are 2.5, 3.0 and 3.5. Here we will be using 3.5

Following a similar approach to the previous methods:

In [ ]:
# Function to find the lower and the upper cutoff values using the Z-Score method

def out_zscore(data):
    global outliers,zscore
    outliers = []
    zscore = []
    threshold = 3.5
    mean = np.mean(data)
    std = np.std(data)
    for i in data:
        z_score= (i - mean)/std 
        zscore.append(z_score)
        if np.abs(z_score) > threshold:
            outliers.append(i)
    return print("Total number of outliers are",len(outliers))


In [ ]:
# Get the 3xSD limits ad the number of outliers
out_zscore(df_wine['pH'])

In [ ]:
df_pH = pd.DataFrame(zscore, columns=['pH'])
df_pH

In [ ]:
# add a column to identify the SD-based outliers
df_wine['Outlier_zscore'] = (df_pH['pH'] < -3.5) | (df_pH['pH'] > 3.5)
pd.crosstab(df_wine['Outlier_zscore'], columns='count')

In [ ]:
# Visualize the univariate ouliers of the dataset
g = sns.JointGrid(data=df_wine, x=df_wine.index, y='pH', hue='Outlier_zscore')
g.plot(sns.scatterplot, sns.boxplot)
g.ax_marg_x.set_axis_off() # removes the marginal x-axis
g.ax_marg_y.set_axis_off() # removes the marginal y-axis
g.ax_marg_x.remove() # removes the marginal x boxplot (index of each observation)
g.set_axis_labels(xlabel='Observation index', ylabel='pH')
g.fig.set_figwidth(8) # to define the width to get a non-square figure with JointGrid 
g.fig.set_figheight(4) # to define the height to get a non-square figure with JointGrid 
plt.show()

In [ ]:
df_wine

#### 2.4 Isolation Forest

Isolation forest is based recursive partitioning of the dataset using a set of decision trees and provides an anomaly score looking at how isolated the point is in the structure found. The anomaly score is then used to detect outliers. The number of splittings required to isolate a sample is equivalent to the path length from the root node to the terminating node. THis method is based on the assumption that it requires fewer splits to isolate an outlier than it does to isolate a non-outlier, i.e. an outlier has a lower isolation number in comparison to a non-outlier observation. 

An observation is therefore defined as an outlier if its isolation number is lower than a given threshold. The threshold is defined based on the estimated percentage of outliers in the data, which is the starting point of this outlier detection algorithm.

Isolation Forest does not assume normal distribution and is able to detect outliers at a multi-dimensional level. Moreover, the algorithm has a linear time complexity with a low constant and a low memory requirement which makes it computationally efficient. Hence, it scales well to large data sets.

The method is implemented in the scikit-learn module: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html

In the following code, examples of how to identify ***univariate, bivariate and multivariate outliers*** are provided.

In [ ]:
#Import necessary libraries
from sklearn.ensemble import IsolationForest

##### 2.4.1 Run a univariate isolation forest (to detect univariate outliers)

Let's run a univariate isolation forest for 'pH'

In [ ]:
model = IsolationForest(n_estimators = 150, contamination='auto') # define the model (150 isolation trees, default: only 1 variable (feature) of the dataset)
model.fit(df_wine['pH'].values.reshape(-1,1)) # fit the model (univariate: 'pH')
scores = model.decision_function(df_wine['pH'].values.reshape(-1,1)) #  extract the anomaly scores (mean anomaly scores of each tree)
anomaly = model.predict(df_wine['pH'].values.reshape(-1,1)) # extract an anomaly (outlier) identifier (-1)
df_wine['scores'] = scores # add a new column to the database with the anomaly scores
df_wine['anomaly'] = anomaly # add a new column to the database with the anomaly identifier (1/-1)
pd.crosstab(df_wine['anomaly'], columns='count') # count the number of outliers

Visualize the univariate ouliers of the dataset

In [ ]:
g = sns.JointGrid(data=df_wine, x=df_wine.index, y='pH', hue='anomaly')
g.plot(sns.scatterplot, sns.boxplot)
g.ax_marg_x.set_axis_off() # removes the marginal x-axis
g.ax_marg_y.set_axis_off() # removes the marginal y-axis
g.ax_marg_x.remove() # removes the marginal x boxplot (index of each observation)
g.set_axis_labels(xlabel='Observation index', ylabel='pH')
g.fig.set_figwidth(8) # to define the width to get a non-square figure with JointGrid 
g.fig.set_figheight(4) # to define the height to get a non-square figure with JointGrid 
plt.show()

##### 2.4.2 Run a bivariate isolation forest

Let's now run a bivariate isolation forest for'pH' and 'citric acid'

In [ ]:
model = IsolationForest(n_estimators = 150, contamination='auto', max_features=2) # define the model (150 isolation trees, default: only 1 variable (feature) of the dataset)
model.fit(df_wine[['citric acid', 'pH']].values) # fit the model (univariate: 'pH')
scores = model.decision_function(df_wine[['citric acid', 'pH']].values) #  extract the anomaly scores (mean anomaly scores of each tree)
anomaly = model.predict(df_wine[['citric acid', 'pH']].values) # extract an anomaly (outlier) identifier (-1)
df_wine['scores'] = scores # add a new column to the database with the anomaly scores
df_wine['anomaly'] = anomaly # add a new column to the database with the anomaly identifier (1/-1)
pd.crosstab(df_wine['anomaly'], columns='count') # count the number of outliers

Visualize the bivariate ouliers of the dataset

In [ ]:
g = sns.JointGrid(data=df_wine, x='citric acid', y='pH', hue='anomaly') 
g.plot(sns.scatterplot, sns.histplot) # to plot both a scatter plot and a boxplot for each variable
g.ax_marg_x.set_axis_off() # removes the marginal x-axis
g.ax_marg_y.set_axis_off() # removes the marginal y-axis
g.fig.set_figwidth(7) # to define the width to get a non-square figure with JointGrid 
g.fig.set_figheight(4) # to define the height to get a non-square figure with JointGrid 
plt.show()

Alternative using continuous anomaly scores instead (may take a while).

In [ ]:
g = sns.JointGrid(data=df_wine, x='citric acid', y='pH', hue='scores') 
g.plot(sns.scatterplot, sns.histplot) # to plot both a scatter plot and a boxplot for each variable
g.ax_marg_x.set_axis_off() # removes the marginal x-axis
g.ax_marg_y.set_axis_off() # removes the marginal y-axis
g.fig.set_figwidth(7) # to define the width to get a non-square figure with JointGrid 
g.fig.set_figheight(4) # to define the height to get a non-square figure with JointGrid 
plt.show()

##### 2.4.3 Run a multivariate isolation forest

Let's now run a bivariate isolation forest for all variables in the dataset (including categorical)

In [ ]:
model = IsolationForest(n_estimators = 150, contamination='auto', max_features=12) # define the model (150 isolation trees, all 12 variables in the dataset)
model.fit(df_wine.values) # fit the model to the data
scores = model.decision_function(df_wine.values) # extract the anomaly scores (mean anomaly scores of each tree)
anomaly = model.predict(df_wine.values) # to extract an anomaly (outlier) identifier (-1)
df_wine['scores'] = scores # add a new column to the database with the anomaly scores
df_wine['anomaly'] = anomaly # add a new column to the database with the anomaly identifier(1/-1)
pd.crosstab(df_wine['anomaly'], columns='count') # count the number of outliers

Visualize the multivariate ouliers of the dataset using a univariate plot for 'pH' (just as an example).

NOTE: It is dificult to visualize outliers for more than 2 dimensions (maybe a biplot of a Principal Component Analysis would help).

In [ ]:
g = sns.JointGrid(data=df_wine, x=df_wine.index, y='pH', hue='anomaly')
g.plot(sns.scatterplot, sns.boxplot)
g.ax_marg_x.set_axis_off() # removes the marginal x-axis
g.ax_marg_y.set_axis_off() # removes the marginal y-axis
g.ax_marg_x.remove() # removes the marginal x boxplot (index of each observation)
g.set_axis_labels(xlabel='Observation index', ylabel='pH')
g.fig.set_figwidth(8) # to define the width to get a non-square figure with JointGrid 
g.fig.set_figheight(4) # to define the height to get a non-square figure with JointGrid 
plt.show()

### Conclusion

The different detection methods differ greatly leading to marked differences in outlier detection. It is important to detect outliers by finding consensus among different methods. After detecting outliers, it important to assess whether they correspond to real data as well as to evaluate their impact on the analytical methods we intend to use. Based on this assessment, we may use different approaches to deal with them.

There are many other methods for outlier detection processes that you may search for, such as:
- DBSCAN (Density-Based Spatial Clustering of Applications with Noise)
- Local Outlier Factor Method (LOF):
- Elliptic Envelope
- One-Class Support Vector Machines
- Robust Random Cut Forest

### References

https://www.analyticsvidhya.com/blog/2021/06/univariate-anomaly-detection-a-walkthrough-in-python/

https://www.kaggle.com/code/rpsuraj/outlier-detection-techniques-simplified

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html

